In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd

nfb = pd.read_csv('./drive/My Drive/Мастерская/multilabel_classification_df.csv')

In [ ]:
!pip install transformers

In [ ]:
from IPython.display import clear_output 
import gc
import numpy as np
import re

In [ ]:
import torch
import torch.nn as nn
import transformers
from transformers import (
    AutoModel, AutoConfig, 
    AutoTokenizer, logging
)
logging.set_verbosity_error()
logging.set_verbosity_warning()

max_seq_length = 16
_pretrained_model = 'sberbank-ai/ruBert-base'

config = AutoConfig.from_pretrained(_pretrained_model)
model = AutoModel.from_pretrained(_pretrained_model, config=config)
tokenizer = AutoTokenizer.from_pretrained(_pretrained_model)

Downloading:   0%|          | 0.00/590 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/683M [00:00<?, ?B/s]

Some weights of the model checkpoint at sberbank-ai/ruBert-base were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/1.70M [00:00<?, ?B/s]

In [ ]:
clear_output()

In [ ]:
nfb = nfb.dropna()

In [ ]:
train_text = nfb['Example'].to_list()

In [ ]:
def trying(tokenizer, train_text, max_seq_length, config, model):
  features = tokenizer.batch_encode_plus(
    train_text,
    add_special_tokens=True,
    padding='max_length',
    max_length=max_seq_length,
    truncation=True,
    return_tensors='pt',
    return_attention_mask=True
 )

  with torch.no_grad():
    outputs = model(features['input_ids'], features['attention_mask'])
  last_hidden_state = outputs[0]
  cls_embeddings = last_hidden_state[:, 0]
  logits = nn.Linear(config.hidden_size, 1)(cls_embeddings) # regression head

#   print(f'Last Hidden State Output Shape: {last_hidden_state.detach().numpy().shape}')
#   print(f'CLS Embeddings Output Shape: {cls_embeddings.detach().numpy().shape}')
#   print(f'Logits Shape: {logits.detach().numpy().shape}')

  del config, model, tokenizer, features, outputs
#   gc.collect();
  return cls_embeddings, last_hidden_state, logits

In [ ]:
cls_embeddings = []
last_hidden_state = []
logits = []

In [ ]:
from tqdm import tqdm

for text in tqdm(train_text):
    embd = trying(tokenizer, [text], max_seq_length, config, model)
    cls_embeddings.append(embd[0])
    last_hidden_state.append(embd[1])
    logits.append(embd[2])
    del embd

100%|██████████| 9796/9796 [16:08<00:00, 10.12it/s]


In [ ]:
nfb['vecs'] = [np.array(t, dtype=float)[0] for t in cls_embeddings]

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_auc_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn import model_selection
import sklearn.metrics as metrics


In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

In [ ]:
y = nfb['arg_num']
x = nfb['vecs']

X_train, X_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, random_state=1337)

log_model = LogisticRegression(solver='lbfgs', max_iter=3000)
# model1 = Pipeline([('log_reg', lr)])
log_model.fit(list(X_train), y_train)
# model1.fit(list(X_train), y_train)
y_pred = log_model.predict(list(X_valid))
# y_pred = model1.predict(list(X_valid))

print("Accuracy:",metrics.accuracy_score(y_valid, y_pred))
print("Precision:",metrics.precision_score(y_valid, y_pred, average='weighted'))
print("Recall:",metrics.recall_score(y_valid, y_pred, average='weighted'))

Accuracy: 0.601530612244898
Precision: 0.5713308950580528
Recall: 0.601530612244898


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
len_sentences = [len(re.findall(r'\w+', text)) for text in nfb['Example']]
nfb['len_sentences'] = len_sentences

In [ ]:
nfb = nfb.reset_index(drop=True)

In [ ]:
verbs_vec = []
nargs = []
vecs = []
len_sentences = []

for i in range(len(nfb)):
    vec = tokenizer.encode_plus(nfb['Lex'][i], return_tensors='pt')['input_ids'][0]
    if len(vec) == 3:
        verbs_vec.append(vec[1])
        nargs.append(nfb.at[nfb.index[i],'arg_num'])
        vecs.append(nfb.at[nfb.index[i],'vecs'])
        len_sentences.append(nfb.at[nfb.index[i],'len_sentences'])

In [ ]:
onemore = pd.DataFrame({'lex_vec': verbs_vec,
                        'arg_num': nargs,
                        'vecs': vecs,
                        'len_sentences': len_sentences})

In [ ]:
onemore['lex_vec'] = [np.array(t, dtype=float) for t in onemore['lex_vec']]

In [ ]:
nfb.to_csv('nfb_new.csv')

In [ ]:
onemore.to_csv('onemore.csv')

In [ ]:
onemore.head(15)

In [ ]:
onemore['vecs'] = [x.astype(np.float) for x in onemore['vecs']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.


In [ ]:
onemore['average_vecs'] = [np.average(x) for x in onemore['vecs']]
y = onemore['arg_num']
x = onemore[['average_vecs', 'len_sentences', 'lex_vec']]

X_train, X_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, random_state=1337)

lr = LogisticRegression(solver='lbfgs', max_iter=5000)
# # # # log_model = LogisticRegression(solver='lbfgs', max_iter=3000)
model1 = Pipeline([('log_reg', lr)])
# # # # log_model.fit(X_train, y_train)
model1.fit(X_train, y_train)
# # # y_pred = log_model.predict(X_valid)
y_pred = model1.predict(X_valid)

print("Accuracy:",metrics.accuracy_score(y_valid, y_pred))
print("Precision:",metrics.precision_score(y_valid, y_pred, average='weighted'))
print("Recall:",metrics.recall_score(y_valid, y_pred, average='weighted'))


Accuracy: 0.6427556818181818
Precision: 0.4131348665095558
Recall: 0.6427556818181818


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
